In [4]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Model, Dataset
from azureml.data.datapath import DataPath
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.57.0 to work with azuremlws


In [14]:
datastore = ws.get_default_datastore()
datastore.upload_files(['./diabetes.csv'], target_path = 'diabetes/')
datastore_path = 'diabetes/diabetes.csv'
datapath = DataPath(datastore, datastore_path)
ds = Dataset.Tabular.from_delimited_files(path = datapath)

ds = ds.register(
    workspace = ws,
    name = 'DIABETES',
    description = 'diabetes classification',
    create_new_version = True
)

ds.name

Uploading an estimated of 1 files
Uploading ./diabetes.csv
Uploaded ./diabetes.csv, 1 files out of an estimated total of 1
Uploaded 1 files


'DIABETES'

In [5]:
ds = Dataset.get_by_name(workspace=ws, name='DIABETES')
df = ds.to_pandas_dataframe()
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,True
1,1,85,66,29,0,26.6,0.351,31,False
2,8,183,64,0,0,23.3,0.672,32,True
3,1,89,66,23,94,28.1,0.167,21,False
4,0,137,40,35,168,43.1,2.288,33,True


In [13]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
curated_env = 'AzureML-sklearn-1.0-ubuntu20.04-py38-cpu'
env = Environment.get(workspace=ws, name=curated_env)

# Create a script config
script_config = ScriptRunConfig(source_directory='.',
                                script='training.py',
                                environment=env) 

# submit the experiment run
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# Block until the experiment run has completed
run.wait_for_completion()

{'runId': 'mslearn-train-diabetes_1737055039_814f3cba',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2025-01-16T19:17:22.59144Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  '_azureml.ClusterName': 'local',
  'ContentSnapshotId': '565403e3-c8ca-474c-89b3-fa51f76096fa',
  'azureml.git.repository_uri': 'https://github.com/mayureshwagh/AzureML.git',
  'mlflow.source.git.repoURL': 'https://github.com/mayureshwagh/AzureML.git',
  'azureml.git.branch': 'mayuresh_feature',
  'mlflow.source.git.branch': 'mayuresh_feature',
  'azureml.git.commit': '27bbaf511dfc26515cac7732cde0778f8c225a46',
  'mlflow.source.git.commit': '27bbaf511dfc26515cac7732cde0778f8c225a46',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'loca

In [14]:
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

AUC 0.8357720778102944
Accuracy 0.7792207792207793
Regularization Rate 0.01


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/6813_azureml.log
outputs/diabetes_model.pkl


In [16]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8357720778102944
	 Accuracy : 0.7792207792207793




In [18]:
for model in Model.list(ws):
    print(f'Model:{model.name}, Version:{model.version}')

Model:diabetes_model, Version:1


In [23]:
from azureml.core import Workspace, Environment, Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core import Environment

inference_config = InferenceConfig(entry_script="scoring.py", environment=env)

# 5. Deploy to ACI (Azure Container Instance) for testing or to AKS (Azure Kubernetes Service) for production
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

model = Model(ws, name="diabetes_model", version=1)
# 6. Deploy the model to ACI
service = Model.deploy(
    workspace=ws,
    name="model-inference-service",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)

# Wait for deployment to complete
service.wait_for_deployment(show_output=True)

# 7. Get the scoring URL of the deployed service
print(service.scoring_uri)

/tmp/ipykernel_6121/1585736783.py:14: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-01-16 19:36:10+00:00 Creating Container Registry if not exists.

In [37]:
from azureml.core.webservice import Webservice
service = Webservice(workspace = ws, name='model-inference-service')

# Print details
print("Service State:", service.state)
print("Scoring URI:", service.scoring_uri)
service.update(enable_app_insights = True)

Service State: Healthy
Scoring URI: http://eb26ccdf-623c-459f-9015-cf596249b39d.eastus.azurecontainer.io/score


In [36]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] diabetic


In [40]:
service.delete()
print ('Service deleted.')

No service with name model-inference-service found to delete.
Service deleted.
